<font size="+3"><strong>2. Extract, Transform, Load</strong></font>

In the last notebook, we focused on exploratory data analysis. Specifically, we extracted information from our MongoDB database in order to describe some characteristics of the DS Lab applicant pool — country of origin, age, and education level. In this notebook, our goal is to design our experiment, and that means we'll need to go beyond extracting information. We'll also need to make some transformations in our data and then load it back into our database.

In Data Science and Data Engineering, the process of taking data from a source, changing it, and then loading it into a database is called **ETL**, which is short for **extract, transform, load**. ETL tends to be more programming-intensive than other data science tasks like visualization, so we'll also spend time in this lesson exploring Python as an **object-oriented programming** language. Specifically, we'll create our own Python **class** to contain our ETL processes.

In [4]:
# Import libraries
import random

import pandas as pd
from pymongo.mongo_client import MongoClient

# **1. Connect**

In [12]:
client = MongoClient(uri)
db = client["ds-abtest"]
ds_app = db["ds-applicants"]

print("client:", type(client))
print("ds_app:", type(ds_app))

client: <class 'pymongo.mongo_client.MongoClient'>
ds_app: <class 'pymongo.collection.Collection'>


# **2. Extract: Developing the Hypothesis**

Now that we've connected to the data, we need to pull out the information we need. One aspect of our applicant pool that we didn't explore in the last notebook is how many applicants actually complete the DS python quiz.

In [14]:
# How many applicants complete python quiz?
result = ds_app.aggregate(
    [
        {
            "$group": {
                 "_id": "$pythonQuiz",
                 "count": {"$count":{}}
                }
            }
        ]
    )

for r in result:
    if r["_id"] == "incomplete":
        incomplete = r["count"]
    else:
        complete = r["count"]

print("Completed quiz:", complete)
print("Did not complete quiz:", incomplete)

Completed quiz: 971
Did not complete quiz: 364


That gives us some raw numbers, but we're interested in participation *rates*, not participation numbers. Let's turn what we just got into a percentage.

In [ ]:
total = complete + incomplete
prop_incomplete = incomplete / total
print(
    "Proportion of users who don't complete admissions quiz:", round(prop_incomplete, 2)
)

Proportion of users who don't complete admissions quiz: 0.27


Now that we know that around a quarter of DS Lab applicants don't complete the admissions quiz, is there anything we can do improve the completion rate?


A **hypothesis** is an informed guess about what we think is going to happen in an experiment. We probably hope that whatever we're trying out is going to work, but it's important to maintain a healthy degree of skepticism. Science experiments are designed to demonstrate what *does* work, not what doesn't, so we always start out by assuming that whatever we're about to do won't make a difference (even if we hope it will). The idea that an experimental intervention won't change anything is called a **null hypothesis** ($H_0$), and every experiment either rejects the null hypothesis (meaning the intervention worked), or fails to reject the null hypothesis (meaning it didn't).

The mirror image of the null hypothesis is called an **alternate hypothesis** ($H_a$), and it proceeds from the idea that whatever we're about to do actually *will* work. If I'm trying to figure out whether exercising is going to help me lose weight, the null hypothesis says that if I exercise, I won't lose any weight. The alternate hypothesis says that if I exercise, I will lose weight.

It's important to keep both types of hypothesis in mind as we work through our experimental design.

In [15]:
null_hypothesis = """
   There is no relationship between receiving an email and completing the python quiz.
   Sending an email to "no-quiz applicants" doesn't increase the rate of completion.
"""
alternate_hypothesis = """
   There is a relationship between receiving an email and completing the python quiz.
   Sending an email to 'no-quiz applicants' does increase the rate of completion.
"""
print("Null Hypothesis:", null_hypothesis)
print("Alternate Hypothesis:", alternate_hypothesis)

Null Hypothesis: 
   There is no relationship between receiving an email and completing the python quiz.
   Sending an email to "no-quiz applicants" doesn't increase the rate of completion.

Alternate Hypothesis: 
   There is a relationship between receiving an email and completing the python quiz.
   Sending an email to 'no-quiz applicants' does increase the rate of completion.



The next thing we need to do is figure out a way to filter the data so that we're only looking at students who applied on a certain date. This is a perfect chance to write a function!

In [16]:
def find_by_date(collection, date_string):
    """Find records in a PyMongo Collection created on a given date.

    Parameters
    ----------
    collection : pymongo.collection.Collection
        Collection in which to search for documents.
    date_string : str
        Date to query. Format must be '%Y-%m-%d', e.g. '2022-06-28'.

    Returns
    -------
    observations : list
        Result of query. List of documents (dictionaries).
    """
    # Convert `date_string` to datetime object
    start = pd.to_datetime(date_string, format = "%Y-%m-%d")
    # Offset `start` by 1 day
    end = start + pd.DateOffset(days=1)
    # Create PyMongo query for no-quiz applicants b/t `start` and `end`
    query = {"createdAt": {"$gte": start, "$lt": end}, "pythonQuiz": "incomplete"}
    # Query collection, get result
    result = collection.find(query)
    # Convert `result` to list
    observations = list(result)
    # REMOVE}
    return observations

3 June 2022 seems like as good a date as any, so let's use the function we just wrote to get all the students who applied that day.

In [18]:
observations = find_by_date(ds_app, date_string="2022-06-03")

print("observations type:", type(observations))
print("observations len:", len(observations))
observations[0]

observations type: <class 'list'>
observations len: 12


{'_id': ObjectId('651c300236eab9d3c09aeede'),
 'createdAt': datetime.datetime(2022, 6, 3, 16, 38, 11),
 'firstName': 'James',
 'lastName': 'Huston',
 'email': 'james.huston13@gmall.com',
 'birthday': datetime.datetime(1996, 4, 21, 0, 0),
 'gender': 'male',
 'highestDegreeEarned': "Bachelor's degree",
 'countryISO2': 'IN',
 'pythonQuiz': 'incomplete'}

# **3. Transform: Designing the Experiment**

Okay! Now that we've extracted the data we'll need for the experiment, it's time to get our hands dirty.

The **transform** stage of ETL involves manipulating the data we just extracted. In this case, we're going to be figuring out which students didn't take the quiz, and assigning them to different experimental groups. To do that, we'll need to *transform* each document in the database by creating a new attribute for each record.

Now we can split the students who didn't take the quiz into two groups: one that will receive a reminder email, and one that will not.

In [19]:
def assign_to_groups(observations):
    """Randomly assigns observations to control and treatment groups.

    Parameters
    ----------
    observations : list or pymongo.cursor.Cursor
        List of users to assign to groups.

    Returns
    -------
    observations : list
        List of documents from `observations` with two additional keys:
        `inExperiment` and `group`.
    """
    # Shuffle `observations`
    random.seed(42)
    random.shuffle(observations)

    # Get index position of item at observations halfway point
    idx = len(observations) // 2

    # Assign first half of observations to control group
    for doc in observations [:idx]:
        doc["inExperiment"] = True
        doc["group"] = "no email (control)"

    # Assign second half of observations to treatment group
    for doc in observations [idx:]:
        doc["inExperiment"] = True
        doc["group"] = "email (treatment)"

    return observations

In [20]:
observations_assigned = assign_to_groups(observations)

print("observations_assigned type:", type(observations_assigned))
print("observations_assigned len:", len(observations_assigned))
observations_assigned[0]

observations_assigned type: <class 'list'>
observations_assigned len: 12


{'_id': ObjectId('651c300236eab9d3c09af0b0'),
 'createdAt': datetime.datetime(2022, 6, 3, 19, 33, 44),
 'firstName': 'Armando',
 'lastName': 'Kozel',
 'email': 'armando.kozel49@gmall.com',
 'birthday': datetime.datetime(1993, 2, 15, 0, 0),
 'gender': 'male',
 'highestDegreeEarned': "Bachelor's degree",
 'countryISO2': 'US',
 'pythonQuiz': 'incomplete',
 'inExperiment': True,
 'group': 'no email (control)'}

In [21]:
def export_treatment_emails(observations_assigned, directory="."):
    """Creates CSV file with email addresses of observations in treatment group.

    CSV file name will include today's date, e.g. `'2022-06-28_ab-test.csv'`,
    and a `'tag'` column where every row will be 'ab-test'.

    Parameters
    ----------
    observations_assigned : list
        Observations with group assignment.
    directory : str, default='.'
        Location for saved CSV file.

    Returns
    -------
    None
    """
    # Put `observations_assigned` docs into DataFrame
    df = pd.DataFrame(observations_assigned)

    # Add `"tag"` column
    df["tag"] = "ab-test"

    # Create mask for treatment group only
    mask = df["group"] == "email (treatment)"

    # Create filename with date
    date_string = pd.Timestamp.now().strftime(format="%Y-%m-%d")
    filename = directory + "/" + date_string + "_ab-test.csv"

    # Save DataFrame to directory (email and tag only)
    df[mask][["email", "tag"]].to_csv(filename, index=False)

In [22]:
export_treatment_emails(observations_assigned=observations_assigned)

# **4. Load: Preparing the Data**

We've *extracted* the data and written a bunch of functions we can use to *transform* the data, so it's time for the third part of this module: *loading* the data.

We've assigned the no-quiz applicants to groups for our experiment, so we should update the records in the `"ds-applicants"` collection to reflect that assignment. Before we update all our records, let's start with just one.

In [23]:
# Assign the first item in observations_assigned list to the variable updated_applicant. The assign that applicant's ID to the variable applicant_id.
updated_applicant = observations_assigned[0]
applicant_id = updated_applicant["_id"]
print("applicant type:", type(updated_applicant))
print(updated_applicant)
print()
print("applicant_id type:", type(applicant_id))
print(applicant_id)

applicant type: <class 'dict'>
{'_id': ObjectId('651c300236eab9d3c09af0b0'), 'createdAt': datetime.datetime(2022, 6, 3, 19, 33, 44), 'firstName': 'Armando', 'lastName': 'Kozel', 'email': 'armando.kozel49@gmall.com', 'birthday': datetime.datetime(1993, 2, 15, 0, 0), 'gender': 'male', 'highestDegreeEarned': "Bachelor's degree", 'countryISO2': 'US', 'pythonQuiz': 'incomplete', 'inExperiment': True, 'group': 'no email (control)'}

applicant_id type: <class 'bson.objectid.ObjectId'>
651c300236eab9d3c09af0b0


Now that we have the unique identifier for one of the applicants, we can find it in the collection.

In [24]:
# Find original record for `applicant_id`
ds_app.find_one({"_id": applicant_id})

{'_id': ObjectId('651c300236eab9d3c09af0b0'),
 'createdAt': datetime.datetime(2022, 6, 3, 19, 33, 44),
 'firstName': 'Armando',
 'lastName': 'Kozel',
 'email': 'armando.kozel49@gmall.com',
 'birthday': datetime.datetime(1993, 2, 15, 0, 0),
 'gender': 'male',
 'highestDegreeEarned': "Bachelor's degree",
 'countryISO2': 'US',
 'pythonQuiz': 'incomplete'}

And now we can update that document to show which group that applicant belongs to.

In [25]:
result = ds_app.update_one(
    filter={"_id": applicant_id},
    update={"$set": updated_applicant}
)
print("result type:", type(result))

result type: <class 'pymongo.results.UpdateResult'>


We know how to update a record, and we can interpret our operation results. Since we can do it for one record, we can do it for all of them! So let's update the records for all the observations in our experiment.

In [27]:
def update_applicants(collection, observations_assigned):
    """Update applicant documents in collection.

    Parameters
    ----------
    collection : pymongo.collection.Collection
        Collection in which documents will be updated.

    observations_assigned : list
        Documents that will be used to update collection

    Returns
    -------
    transaction_result : dict
        Status of update operation, including number of documents
        and number of documents modified.
    """
    # Initialize counters
    n = 0
    n_modified = 0

    # Iterate through applicants
    for doc in observations_assigned:
        # Update doc
        result = collection.update_one(
            filter={"_id": doc["_id"]},
            update={"$set": doc}
        )
        # Update counters
        n += result.matched_count
        n_modified += result.modified_count

    # Create results
    transaction_result = {"n": n, "nModified": n_modified}

    return transaction_result

In [28]:
result = update_applicants(ds_app, observations_assigned)
print("result type:", type(result))
result

result type: <class 'dict'>


{'n': 12, 'nModified': 11}

# **5. Python Classes: Building the Repository**

We've managed to extract data from our database using our `find_by_date` function, transform it using our `assign_to_groups` function, and load it using our `update_applicants` function. Does that mean we're done? Not yet! There's an issue we need to address: distraction.

What do we mean when we say distraction? Think about it this way: Do you need to know the exact code that makes `df.describe()` work? No, you just need to calculate summary statistics! Going into more details would distract you from the work you need to get done. The same is true of the tools you've created in this lesson. Others will want to use them in future experiments with worrying about your implementation. The solution is to **abstract** the details of your code away.

To do this we're going to create a [Python class.](https://docs.python.org/3/tutorial/classes.html) Python classes contain both information and ways to interact with that information. An example of class is a pandas `DataFrame`. Not only does it hold data (like the size of an apartment in Buenos Aires or the income of a household in the United States); it also provides methods for inspecting it (like `DataFrame.head()` or `DataFrame.info()`) and manipulating it (like `DataFrame.sum()` or `DataFrame.replace()`).

In the case of this project, we want to create a class that will hold information about the documents we want (like the name and location of the collection) and provide tools for interacting with those documents (like the functions we've built above).

In [29]:
class MongoRepository:
    """Repository class for interacting with MongoDB database.

    Parameters
    ----------
    client : `pymongo.MongoClient`
        By default, `MongoClient(host='localhost', port=27017)`.
    db : str
        By default, `'wqu-abtest'`.
    collection : str
        By default, `'ds-applicants'`.

    Attributes
    ----------
    collection : pymongo.collection.Collection
        All data will be extracted from and loaded to this collection.
    """

    # Task 7.2.14
    def __init__(
        self,
        client = MongoClient(uri),
        db = client["ds-abtest"],
        ds_app = db["ds-applicants"]

     ):
        self.collection = db["ds-applicants"]

    def find_by_date(self, date_string):
        # Convert `date_string` to datetime object
        start = pd.to_datetime(date_string, format = "%Y-%m-%d")
        # Offset `start` by 1 day
        end = start + pd.DateOffset(days=1)
        # Create PyMongo query for no-quiz applicants b/t `start` and `end`
        query = {"createdAt": {"$gte": start, "$lt": end}, "pythonQuiz": "incomplete"}
        # Query collection, get result
        result = self.collection.find(query)
        # Convert `result` to list
        observations = list(result)

        return observations

    def update_applicants(self, observations_assigned):

        # Initialize counters
        n = 0
        n_modified = 0

        # Iterate through applicants
        for doc in observations_assigned:

            # Update doc
            result = self.collection.update_one(
                  filter={"_id": doc["_id"]},
                  update={"$set": doc}
            )
            # Update counters
            n += result.matched_count
            n_modified += result.modified_count

        # Create results
        transaction_result = {"n": n, "nModified": n_modified}

        return transaction_result

    def assign_to_groups(self, date_string):

        #Get observations
        observations = self.find_by_date(date_string)

        # Shuffle `observations`
        random.seed(42)
        random.shuffle(observations)

        # Get index position of item at observations halfway point
        idx = len(observations) // 2

        # Assign first half of observations to control group
        for doc in observations [:idx]:
            doc["inExperiment"] = True
            doc["group"] = "no email (control)"

        # Assign second half of observations to treatment group
        for doc in observations [idx:]:
            doc["inExperiment"] = True
            doc["group"] = "email (treatment)"

        # Update collection
        result = self.update_applicants(observations)
        return result

Now that we have a class definition, we can do all sorts of interesting things. The first thing to do is instantiate the class...

In [30]:
repo = MongoRepository()
print("repo type:", type(repo))
repo

repo type: <class '__main__.MongoRepository'>


...and then we can look at the attributes of the collection.

In [31]:
dir(repo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'assign_to_groups',
 'collection',
 'find_by_date',
 'update_applicants']

In [33]:
c_test = repo.collection
print("c_test type:", type(c_test))

c_test type: <class 'pymongo.collection.Collection'>


In [35]:
june_16_users = repo.find_by_date(date_string="2022-06-16")
print("june_16_users type", type(june_16_users))
print("june_16_users len", len(june_16_users))
june_16_users[:3]

june_15_users type <class 'list'>
june_15_users len 11


[{'_id': ObjectId('651c300236eab9d3c09aeef3'),
  'createdAt': datetime.datetime(2022, 6, 16, 23, 31, 48),
  'firstName': 'Michael',
  'lastName': 'Anaya',
  'email': 'michael.anaya90@yahow.com',
  'birthday': datetime.datetime(1997, 10, 15, 0, 0),
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'countryISO2': 'NG',
  'pythonQuiz': 'incomplete'},
 {'_id': ObjectId('651c300236eab9d3c09aeef9'),
  'createdAt': datetime.datetime(2022, 6, 16, 9, 39, 12),
  'firstName': 'John',
  'lastName': 'Rivera',
  'email': 'john.rivera5@microsift.com',
  'birthday': datetime.datetime(1998, 12, 29, 0, 0),
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'countryISO2': 'NG',
  'pythonQuiz': 'incomplete'},
 {'_id': ObjectId('651c300236eab9d3c09aeff3'),
  'createdAt': datetime.datetime(2022, 6, 16, 15, 33, 18),
  'firstName': 'Steven',
  'lastName': 'Camarillo',
  'email': 'steven.camarillo62@microsift.com',
  'birthday': datetime.datetime(1993, 6, 28, 0, 0),
  'gender

In [36]:
result = repo.update_applicants(observations_assigned)
print("result type:", type(result))
result

result type: <class 'dict'>


{'n': 12, 'nModified': 0}

In [37]:
result = repo.assign_to_groups(date_string="2022-06-17")
print("result type:", type(result))
result

result type: <class 'dict'>


{'n': 12, 'nModified': 12}

In [40]:
repo_test = MongoRepository()
repo_test.assign_to_groups("2022-06-20")
repo_test.find_by_date("2022-06-20")

[{'_id': ObjectId('651c300236eab9d3c09aeebb'),
  'createdAt': datetime.datetime(2022, 6, 20, 4, 42, 59),
  'firstName': 'James',
  'lastName': 'Smith',
  'email': 'james.smith16@hotmeal.com',
  'birthday': datetime.datetime(2003, 10, 9, 0, 0),
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'countryISO2': 'NG',
  'pythonQuiz': 'incomplete',
  'group': 'email (treatment)',
  'inExperiment': True},
 {'_id': ObjectId('651c300236eab9d3c09aeef7'),
  'createdAt': datetime.datetime(2022, 6, 20, 19, 1, 48),
  'firstName': 'Richard',
  'lastName': 'Simons',
  'email': 'richard.simons92@microsift.com',
  'birthday': datetime.datetime(1998, 6, 3, 0, 0),
  'gender': 'male',
  'highestDegreeEarned': "Bachelor's degree",
  'countryISO2': 'TR',
  'pythonQuiz': 'incomplete',
  'group': 'email (treatment)',
  'inExperiment': True},
 {'_id': ObjectId('651c300236eab9d3c09aef4d'),
  'createdAt': datetime.datetime(2022, 6, 20, 5, 21, 34),
  'firstName': 'Mary',
  'lastName': 'Rovero',
 